# 🔍 YOLOv8 - Deteção com Filtragem Manual por IOU (NMS)
Este notebook executa deteção de objetos usando um modelo YOLOv8 treinado, aplicando Non-Maximum Suppression (NMS) manual para eliminar caixas sobrepostas com base no limiar de IOU.

In [5]:
# Instalação necessária (caso ainda não esteja instalada)
!pip install ultralytics opencv-python

In [6]:
from ultralytics import YOLO
import cv2
import os
import torch
import torchvision.ops as ops

## ⚙️ Configurações do modelo e parâmetros

In [ ]:
# Caminho relativo (a partir do local onde está o notebook)
MODELO = 'model/y8s_finetune15/weights/best.pt'

PASTA_IMAGENS = 'chestx_det10_yolo/images/val'

# Thresholds
THRESHOLD = 0.3
IOU_THRESHOLD = 0.4  # Threshold para Non-Maximum Suppression

## 🧮 Fórmula da Interseção sobre União (IOU)
**IOU** mede a sobreposição entre duas bounding boxes. A fórmula é:

$$ IOU = \frac{Área\ da\ Interseção}{Área\ da\ União} $$

Usa-se para decidir se duas caixas estão demasiado próximas e apenas a de maior confiança deve ser mantida.

In [15]:
# Carregar o modelo YOLO treinado
model = YOLO(MODELO)

In [16]:
def filtrar_por_iou(boxes, scores, labels, iou_thresh=0.4):
    if len(boxes) == 0:
        return [], [], []
    keep_idxs = ops.nms(torch.tensor(boxes, dtype=torch.float32),
                       torch.tensor(scores, dtype=torch.float32),
                       iou_thresh)
    boxes_filtradas = [boxes[i] for i in keep_idxs]
    scores_filtrados = [scores[i] for i in keep_idxs]
    labels_filtradas = [labels[i] for i in keep_idxs]
    return boxes_filtradas, scores_filtrados, labels_filtradas

In [ ]:
for nome_ficheiro in os.listdir(PASTA_IMAGENS):
    if nome_ficheiro.lower().endswith(('.jpg', '.jpeg', '.png')):
        caminho_imagem = os.path.join(PASTA_IMAGENS, nome_ficheiro)
        results = model(caminho_imagem)
        img = cv2.imread(caminho_imagem)

        boxes, scores, labels = [], [], []

        for box in results[0].boxes:
            conf = float(box.conf[0])
            if conf >= THRESHOLD:
                cls = int(box.cls[0])
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                boxes.append([x1, y1, x2, y2])
                scores.append(conf)
                labels.append(model.names[cls])

        if boxes:
            print(f"⚠️ Deteções na imagem: {nome_ficheiro}")
            boxes_filtradas, scores_filtrados, labels_filtradas = filtrar_por_iou(
                boxes, scores, labels, iou_thresh=IOU_THRESHOLD)

            for box, conf, nome in zip(boxes_filtradas, scores_filtrados, labels_filtradas):
                x1, y1, x2, y2 = box
                label = f"{nome} {conf:.2f}"
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
                cv2.rectangle(img, (x1, y1 - text_height - 6), (x1 + text_width, y1), (0, 255, 0), -1)
                cv2.putText(img, label, (x1, y1 - 4), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
                print(f" → {nome}: {conf:.2f} - Caixa: {[x1, y1, x2, y2]}")

            cv2.imshow("Deteções", img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

print("✅ Processo concluído.")


image 1/1 c:\Users\lynxv\Desktop\GitHub\projecto-apvc\project-apvc\chestx_det10_yolo\images\train\36200.png: 640x640 1 Fibrosis, 1 Nodule, 220.0ms
Speed: 8.5ms preprocess, 220.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\lynxv\Desktop\GitHub\projecto-apvc\project-apvc\chestx_det10_yolo\images\train\36201.png: 640x640 (no detections), 236.4ms
Speed: 7.8ms preprocess, 236.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\lynxv\Desktop\GitHub\projecto-apvc\project-apvc\chestx_det10_yolo\images\train\36202.png: 640x640 (no detections), 218.5ms
Speed: 7.8ms preprocess, 218.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\lynxv\Desktop\GitHub\projecto-apvc\project-apvc\chestx_det10_yolo\images\train\36203.png: 640x640 (no detections), 216.6ms
Speed: 6.7ms preprocess, 216.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\lynxv\Deskt